# Deep Dive Project 8
## Chicago Type of Crime | Data Extraction

## 1. Import libraries

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

## 2. Read from CSV

In [ ]:
def get_weather_data():
    file_url = "https://raw.githubusercontent.com/CourTeous33/CS547-assignments/refs/heads/main/2001_2024_chicago_weather.csv"
    return pd.read_csv(file_url)

In [ ]:
def get_holiday_data():
    file_url = "https://raw.githubusercontent.com/CourTeous33/CS547-assignments/refs/heads/main/us_bank_holidays_2001_to_present.csv"
    return pd.read_csv(file_url)

In [ ]:
def get_data():
    file_url = "https://raw.githubusercontent.com/CourTeous33/CS547-assignments/refs/heads/main/Crimes_2024(For%20testing%20only%2C%20Not%20for%20submission).csv"
    return pd.read_csv(file_url)

## 3. Feature selection

### Label: primiary type of crime
### Features:
1. Hour
2. Day of week
3. Holidays
4. Weather (Daily)
  - Precipitation
  - Snowfall
  - Temperature
5. BLock
6. Location description
7. District
8. Ward
9. Community area
10. Latitude
11. Longitude

In [ ]:
crime_df = get_data()
selected = ['Date','Block','Primary Type', 'Location Description','District','Ward','Community Area','Latitude','Longitude']
crime_df = crime_df[selected]
crime_df['DateTime'] = pd.to_datetime(crime_df['Date'], format="%m/%d/%Y %I:%M:%S %p")
crime_df['Hour'] = crime_df['DateTime'].dt.hour
crime_df['Date'] = crime_df['DateTime'].dt.date

In [ ]:
crime_df.iloc[0:10,:]

,Date,Block,Primary Type,Location Description,District,Ward,Community Area,Latitude,Longitude,DateTime,Hour
0,2024-10-18,051XX S HOMAN AVE,OFFENSE INVOLVING CHILDREN,RESIDENCE,8,14,63,41.799867,-87.708644,2024-10-18 00:00:00,0
1,2024-10-10,091XX S EGGLESTON AVE,MOTOR VEHICLE THEFT,STREET,22,9,73,41.728316,-87.634933,2024-10-10 00:00:00,0
2,2024-10-10,009XX N LAWNDALE AVE,DECEPTIVE PRACTICE,APARTMENT,11,27,23,41.897826,-87.718830,2024-10-10 00:00:00,0
3,2024-10-10,045XX N CLARENDON AVE,CRIMINAL DAMAGE,STREET,19,46,3,41.963880,-87.649971,2024-10-10 00:00:00,0
4,2024-10-10,108XX S CALHOUN AVE,BATTERY,APARTMENT,4,10,51,41.698661,-87.561961,2024-10-10 00:00:00,0
5,2024-10-10,048XX N FRANCISCO AVE,MOTOR VEHICLE THEFT,STREET,20,40,4,41.969389,-87.700489,2024-10-10 00:00:00,0
6,2024-10-09,002XX W 47TH ST,ROBBERY,CTA TRAIN,9,20,37,41.809125,-87.631907,2024-10-09 23:57:00,23
7,2024-10-09,014XX N NOBLE ST,CRIMINAL DAMAGE,STREET,14,1,24,41.907233,-87.662728,2024-10-09 23:50:00,23
8,2024-10-09,037XX N PACIFIC AVE,MOTOR VEHICLE THEFT,STREET,16,38,17,41.947445,-87.826799,2024-10-09 23:47:00,23
9,2024-10-09,080XX S WESTERN AVE,BATTERY,RESIDENCE,8,18,70,41.747091,-87.682778,2024-10-09 23:45:00,23


In [ ]:
holiday_df = get_holiday_data()
holiday_df = holiday_df[['Date', 'Holiday']]
holiday_df.head()

,Date,Holiday
0,2001-01-01,New Year's Day
1,2001-01-22,Martin Luther King Jr. Day
2,2001-02-26,Presidents' Day
3,2001-06-04,Memorial Day
4,2001-06-19,Juneteenth National Independence Day


In [ ]:
weather_df = get_weather_data()
weather_df = weather_df[['DATE', 'PRCP', 'SNOW', 'TAVG']]
weather_df.head()

,DATE,PRCP,SNOW,TAVG
0,2001-01-01,0.0,0.0,-9.4
1,2001-01-02,0.0,0.0,-11.1
2,2001-01-03,0.0,0.0,-7.8
3,2001-01-04,0.0,0.0,-3.9
4,2001-01-05,0.0,0.0,-1.7


## 4. Combining Holiday and Weather datasets

In [ ]:
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'], format="%Y-%m-%d").dt.normalize()
holiday_df.head()

,Date,Holiday
0,2001-01-01,New Year's Day
1,2001-01-22,Martin Luther King Jr. Day
2,2001-02-26,Presidents' Day
3,2001-06-04,Memorial Day
4,2001-06-19,Juneteenth National Independence Day


In [ ]:
def merge_holiday(crime_df, holiday_df):
    crime_df['Date_ymd'] = pd.to_datetime(crime_df['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.normalize()
    crime_df = crime_df.merge(holiday_df[['Date', 'Holiday']], how='left', left_on='Date_ymd', right_on='Date')
    crime_df['is_Holiday'] = crime_df['Holiday'].notna()
    crime_df = crime_df.drop(columns=['Date_y', 'Date_ymd', 'Holiday']).rename(columns={"Date_x": "Date"})
    return crime_df

In [ ]:
crime_df = merge_holiday(crime_df, holiday_df)
crime_df = crime_df.dropna(subset=['Date'])
crime_df['Date'].isna().sum()
crime_df['Date'] = crime_df['Date'].astype('datetime64[ns]')

In [ ]:
labor_day_crimes = crime_df[crime_df['is_Holiday'] == 1]
display(labor_day_crimes)

,Date,Block,Primary Type,Location Description,District,Ward,Community Area,Latitude,Longitude,DateTime,Hour,is_Holiday
26817,2024-09-02,069XX S CLYDE AVE,BATTERY,APARTMENT,3,5,43,41.769223,-87.573939,2024-09-02 23:57:00,23,True
26818,2024-09-02,034XX W LEXINGTON ST,BATTERY,APARTMENT,11,24,27,41.871805,-87.711678,2024-09-02 23:55:00,23,True
26819,2024-09-02,040XX N KEDZIE AVE,OTHER OFFENSE,STREET,17,33,16,41.954579,-87.708036,2024-09-02 23:50:00,23,True
26820,2024-09-02,061XX S WHIPPLE ST,THEFT,STREET,8,15,66,41.781875,-87.699564,2024-09-02 23:50:00,23,True
26821,2024-09-02,063XX S GREEN ST,BATTERY,APARTMENT,7,16,68,41.778680,-87.645991,2024-09-02 23:35:00,23,True
...,...,...,...,...,...,...,...,...,...,...,...,...
201091,2024-01-01,033XX N HARLEM AVE,SEX OFFENSE,APARTMENT,16,38,17,41.940335,-87.806831,2024-01-01 00:00:00,0,True
201092,2024-01-01,059XX W LAKE ST,CRIMINAL DAMAGE,APARTMENT,15,29,25,41.887980,-87.773525,2024-01-01 00:00:00,0,True
201093,2024-01-01,021XX N LECLAIRE AVE,CRIMINAL DAMAGE,RESIDENCE,25,26,19,41.919585,-87.753558,2024-01-01 00:00:00,0,True
201094,2024-01-01,034XX W BEACH AVE,OTHER OFFENSE,RESIDENCE,14,26,23,41.907310,-87.713108,2024-01-01 00:00:00,0,True


In [ ]:
def merge_weather(crime_df, weather_df):

    crime_df = crime_df.merge(weather_df, how='left', on='Date')

    return crime_df

In [ ]:
weather_df['Date'] = pd.to_datetime(weather_df['DATE']).dt.normalize()
crime_df.loc['Date'] = pd.to_datetime(crime_df['Date'])
weather_df = weather_df.drop(columns=['DATE'])
weather_df.head()
crime_df = merge_weather(crime_df, weather_df)
crime_df = crime_df.dropna(subset=['Date']) #Drop NaT
crime_df['Date'] = crime_df['Date'].astype('datetime64[ns]')

In [ ]:
crime_df.head()

,Date,Block,Primary Type,Location Description,District,Ward,Community Area,Latitude,Longitude,DateTime,Hour,is_Holiday,PRCP,SNOW,TAVG
0,2024-10-18,051XX S HOMAN AVE,OFFENSE INVOLVING CHILDREN,RESIDENCE,8,14,63,41.799867,-87.708644,2024-10-18,0,False,0.0,0.0,12.6
1,2024-10-10,091XX S EGGLESTON AVE,MOTOR VEHICLE THEFT,STREET,22,9,73,41.728316,-87.634933,2024-10-10,0,False,0.0,0.0,16.5
2,2024-10-10,009XX N LAWNDALE AVE,DECEPTIVE PRACTICE,APARTMENT,11,27,23,41.897826,-87.71883,2024-10-10,0,False,0.0,0.0,16.5
3,2024-10-10,045XX N CLARENDON AVE,CRIMINAL DAMAGE,STREET,19,46,3,41.96388,-87.649971,2024-10-10,0,False,0.0,0.0,16.5
4,2024-10-10,108XX S CALHOUN AVE,BATTERY,APARTMENT,4,10,51,41.698661,-87.561961,2024-10-10,0,False,0.0,0.0,16.5


## 5. Splitting debugging/working datasets

In [ ]:
def get_dates_after(df, timestamp):
    res_df = df.copy()
    timestamp_dt = pd.to_datetime(timestamp)
    res_df['DateTime'] = pd.to_datetime(res_df['DateTime'], format="%m/%d/%Y %I:%M:%S %p")
    return res_df[res_df['DateTime'] > timestamp_dt]

In [ ]:
crime_df_dubug = get_dates_after(crime_df, '2024-09-01') # Modify here to select date after specific time
display(crime_df_dubug)

,Date,Block,Primary Type,Location Description,District,Ward,Community Area,Latitude,Longitude,DateTime,Hour,is_Holiday,PRCP,SNOW,TAVG
0,2024-10-18,051XX S HOMAN AVE,OFFENSE INVOLVING CHILDREN,RESIDENCE,8,14,63,41.799867,-87.708644,2024-10-18 00:00:00,0,False,0.0,0.0,12.6
1,2024-10-10,091XX S EGGLESTON AVE,MOTOR VEHICLE THEFT,STREET,22,9,73,41.728316,-87.634933,2024-10-10 00:00:00,0,False,0.0,0.0,16.5
2,2024-10-10,009XX N LAWNDALE AVE,DECEPTIVE PRACTICE,APARTMENT,11,27,23,41.897826,-87.71883,2024-10-10 00:00:00,0,False,0.0,0.0,16.5
3,2024-10-10,045XX N CLARENDON AVE,CRIMINAL DAMAGE,STREET,19,46,3,41.96388,-87.649971,2024-10-10 00:00:00,0,False,0.0,0.0,16.5
4,2024-10-10,108XX S CALHOUN AVE,BATTERY,APARTMENT,4,10,51,41.698661,-87.561961,2024-10-10 00:00:00,0,False,0.0,0.0,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28387,2024-09-01,019XX N SAWYER AVE,MOTOR VEHICLE THEFT,STREET,14,26,22,41.917113,-87.708663,2024-09-01 00:01:00,0,False,0.0,0.0,22.3
28388,2024-09-01,011XX W CARROLL AVE,THEFT,PARKING LOT / GARAGE (NON RESIDENTIAL),12,27,28,41.88771,-87.65619,2024-09-01 00:01:00,0,False,0.0,0.0,22.3
28389,2024-09-01,055XX N WINTHROP AVE,DECEPTIVE PRACTICE,APARTMENT,20,48,77,41.982624,-87.657987,2024-09-01 00:01:00,0,False,0.0,0.0,22.3
28390,2024-09-01,056XX N RAVENSWOOD AVE,MOTOR VEHICLE THEFT,STREET,20,40,77,41.984554,-87.674644,2024-09-01 00:01:00,0,False,0.0,0.0,22.3


## 6. Convert timestamp

In [ ]:
# convert date to timestamp
def covert_time(df,col):
    df[col] = pd.to_datetime(df[col],format="%m/%d/%Y %I:%M:%S %p")
    df[col] = df[col].apply(lambda x: x.timestamp())
    return df

# Full dataset
df = covert_time(crime_df,'DateTime')
display(df['Date'].head())
print(len(df))

# Debug dataset
df_dubug = covert_time(crime_df_dubug,'DateTime')
display(df_dubug['Date'].head())
print(len(df_dubug))

,Date
0,2024-10-18
1,2024-10-10
2,2024-10-10
3,2024-10-10
4,2024-10-10


201096


,Date
0,2024-10-18
1,2024-10-10
2,2024-10-10
3,2024-10-10
4,2024-10-10


28392


## 6. Output dataset as pickle file

In [ ]:
def to_pickle(df, name):
    df.to_pickle(name + '.pkl')
    return

to_pickle(df, 'crime_df')
to_pickle(df_dubug, 'crime_df_debug')

## 7. Test reading

In [ ]:
# def read_pkl_from_google(location_pair, **kwargs):
#     (loc,gdrive)=location_pair
#     loc = 'https://drive.google.com/uc?id='+gdrive.split('/')[-2]
#     out=pd.read_pickle(loc, **kwargs)
#     return out

# df_read = read_pkl_from_google(("crime_df.pkl", "https://drive.google.com/file/d/1ubIlIjjevdgWtwwQzou074CVKcouyp2M/view?usp=drive_link"))
# df_debug_read = read_pkl_from_google(("crime_df_debug.pkl", "https://drive.google.com/file/d/1-UjFOS0JjPk1OSV3KPic29GLalFVdV9s/view?usp=drive_link"))